In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx

In [2]:
choice = np.random.permutation([i for i in range(1,68)])
choice = sorted (choice[0:25])
print('choosed data:',choice)

choosed data: [3, 5, 7, 9, 13, 16, 24, 25, 29, 34, 36, 37, 43, 45, 46, 47, 48, 50, 51, 52, 59, 60, 62, 66, 67]


In [3]:
image = []
im_train = []
im_test = []
for index in choice:
    temp = ['PIE/'+str(index)+'/'+str(i+1)+'.jpg' for i in range(0,170)]
    number = round(len(temp)*0.7)
    id_train = np.random.permutation([i for i in range(0,len(temp))])
    id_train = sorted(id_train[0:number])
    train = []
    test = []
    for i in range(0,len(temp)):
        if i in id_train:
            train.append(temp[i])
        else:
            test.append(temp[i])
    im_train.extend(train)
    im_test.extend(test)
    image.extend(temp)

print('image size:',len(image))
print('train image size:',len(im_train))
print('test image size:',len(im_test))


image size: 4250
train image size: 2975
test image size: 1275


In [4]:
self = ['own/processed/'+str(i+1)+'.jpg' for i in range (0,10)]
number = round(len(self)*0.7)
a = np.random.permutation([i for i in range(0,len(self))])
a = sorted(a[0:number])
self_train = []
self_test = []
for i in range(0,len(self)):
    if i in a:
        self_train.append(self[i])
    else:
        self_test.append(self[i])
print('self image size:',len(self))
print('self train image size:',len(self_train))
print('self test image size:',len(self_test))


self image size: 10
self train image size: 7
self test image size: 3


In [5]:
label = np.zeros([2982,1])
train_set = []

whole_train = im_train+self_train
for i in range(len(whole_train)):
    path = whole_train[i]
    path2 = path.split('/')
    image = cv2.imread(path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    train_set.append(image)
    
    if path2[0]=='PIE':
        label[i] = int(path2[1])
    elif path2[0]=='own':
        label[i] = 100
train_set = np.array(train_set)
train_set = train_set.reshape(len(train_set), -1)
label = np.array(label)

test_label = np.zeros([1278,1])
test_set = []

whole_test = im_test+self_test
for i in range(len(whole_test)):
    path = whole_test[i]
    path2 = path.split('/')
    image = cv2.imread(path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    test_set.append(image)
    
    if path2[0]=='PIE':
        test_label[i] = int(path2[1])
    elif path2[0]=='own':
        test_label[i] = 100
test_set = np.array(test_set)
test_set = test_set.reshape(len(test_set), -1)
test_label = np.array(test_label)

print(train_set.shape)
print(test_set.shape)
print(label.shape)
print(test_label.shape)

(2982, 1024)
(1278, 1024)
(2982, 1)
(1278, 1)


In [6]:
def PCA_nD(train_set,k):
    m = train_set.mean(axis=0)
    temp = train_set - m
    
    cov_mat = np.cov(temp.T)
    eigenvalues, eigenvectors = np.linalg.eig(cov_mat)
    eigenvectors = eigenvectors.real
    sorted_result = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[sorted_result]
    eigenvectors = eigenvectors[:, sorted_result]
    eigenvalues = eigenvalues[0:k]
    face = eigenvectors[:,:k]
    return eigenvalues, face

In [7]:
ev80, ef80 = PCA_nD(train_set,80)
train80 = np.dot(train_set, ef80)
test80 = np.dot(test_set, ef80)

In [14]:
ev40, ef40 = PCA_nD(train_set,40)
train40 = np.dot(train_set, ef40)
test40 = np.dot(test_set, ef40)

In [8]:
ev200, ef200 = PCA_nD(train_set,200)
train200 = np.dot(train_set, ef200)
test200 = np.dot(test_set, ef200)

In [9]:
from libsvm.svmutil import *
from numpy import ctypeslib
def SVM_calculation(train_data, train_labels, test_data, test_labels, cost_parameter):
    train_labels_list = train_labels.T[0].tolist()
    train_data_list = train_data.tolist()
    test_labels_list = test_labels.T[0].tolist()
    test_data_list = test_data.tolist()

    training_problem = svm_problem(train_labels_list, train_data_list)

    svm_config = '-t 0 -b 1 -c ' + str(cost_parameter)
    svm_params = svm_parameter(svm_config)

    trained_model = svm_train(training_problem, svm_params)

    predicted_labels, accuracy, _ = svm_predict(test_labels_list, test_data_list, trained_model, '-b 1')

    return accuracy[0]

In [10]:
a=SVM_calculation(train80,label,test80,test_label,1)
b=SVM_calculation(train80,label,test80,test_label,0.1)
c=SVM_calculation(train80,label,test80,test_label,0.01)



..*..*
optimization finished, #iter = 768
nu = 0.000000
obj = -0.000046, rho = -0.670775
nSV = 24, nBSV = 0
Total nSV = 24
..*..*
optimization finished, #iter = 832
nu = 0.000000
obj = -0.000047, rho = -0.712014
nSV = 26, nBSV = 0
Total nSV = 26
..*.*
optimization finished, #iter = 623
nu = 0.000000
obj = -0.000043, rho = -0.418594
nSV = 23, nBSV = 0
Total nSV = 23
..*.*
optimization finished, #iter = 741
nu = 0.000000
obj = -0.000043, rho = -0.691323
nSV = 24, nBSV = 0
Total nSV = 24
.*.*
optimization finished, #iter = 466
nu = 0.000000
obj = -0.000044, rho = -0.783412
nSV = 21, nBSV = 0
Total nSV = 21
..*.*
optimization finished, #iter = 775
nu = 0.000000
obj = -0.000048, rho = -0.662077
nSV = 25, nBSV = 0
..*.*
optimization finished, #iter = 588
nu = 0.000001
obj = -0.000052, rho = 0.217429
nSV = 21, nBSV = 0
Total nSV = 21
.*.*
optimization finished, #iter = 387
nu = 0.000000
obj = -0.000042, rho = -0.019276
nSV = 21, nBSV = 0
Total nSV = 21
.*.*
optimization finished, #iter = 528


.*.*
optimization finished, #iter = 562
nu = 0.000000
obj = -0.000048, rho = 0.937554
nSV = 27, nBSV = 0
.*.*
optimization finished, #iter = 451
nu = 0.000000
obj = -0.000041, rho = 0.201442
nSV = 26, nBSV = 0
Total nSV = 26
..*
optimization finished, #iter = 564
nu = 0.000000
obj = -0.000035, rho = -0.411815
nSV = 24, nBSV = 0
Total nSV = 24
.*.*
optimization finished, #iter = 527
nu = 0.000000
obj = -0.000039, rho = 0.433512
nSV = 24, nBSV = 0
Total nSV = 24
..*
optimization finished, #iter = 512
nu = 0.000000
obj = -0.000044, rho = -0.082820
nSV = 26, nBSV = 0
Total nSV = 26
..*..*
optimization finished, #iter = 912
nu = 0.000000
obj = -0.000044, rho = 0.249602
nSV = 26, nBSV = 0
Total nSV = 26
..*.*
optimization finished, #iter = 763
nu = 0.000000
obj = -0.000044, rho = 0.210674
nSV = 28, nBSV = 0
..*.*
optimization finished, #iter = 695
nu = 0.000000
obj = -0.000031, rho = 0.746934
nSV = 24, nBSV = 0
Total nSV = 24
.*.*
optimization finished, #iter = 535
nu = 0.000000
obj = -0.000

...*.*
optimization finished, #iter = 943
nu = 0.000001
obj = -0.000085, rho = -0.212338
nSV = 24, nBSV = 0
Total nSV = 24
...*.*
optimization finished, #iter = 863
nu = 0.000001
obj = -0.000073, rho = -0.563733
nSV = 25, nBSV = 0
Total nSV = 25
.*.*
optimization finished, #iter = 527
nu = 0.000001
obj = -0.000097, rho = 0.100268
nSV = 25, nBSV = 0
.*.*
optimization finished, #iter = 434
nu = 0.000001
obj = -0.000055, rho = 0.107510
nSV = 26, nBSV = 0
Total nSV = 26
.*.*
optimization finished, #iter = 381
nu = 0.000001
obj = -0.000052, rho = -0.146697
nSV = 19, nBSV = 0
Total nSV = 19
.*
optimization finished, #iter = 343
nu = 0.000001
obj = -0.000050, rho = -0.053780
nSV = 23, nBSV = 0
Total nSV = 23
.*.*
optimization finished, #iter = 502
nu = 0.000001
obj = -0.000053, rho = 0.227692
nSV = 24, nBSV = 0
Total nSV = 24
.*..*
optimization finished, #iter = 648
nu = 0.000001
obj = -0.000054, rho = -0.088694
nSV = 28, nBSV = 0
Total nSV = 28
.*.*
optimization finished, #iter = 623
nu = 0.

.*
optimization finished, #iter = 299
nu = 0.000000
obj = -0.000013, rho = 0.937941
nSV = 15, nBSV = 0
Total nSV = 15
*
optimization finished, #iter = 232
nu = 0.000000
obj = -0.000016, rho = 0.994712
nSV = 14, nBSV = 0
.*
optimization finished, #iter = 288
nu = 0.000000
obj = -0.000024, rho = 2.145617
nSV = 20, nBSV = 0
Total nSV = 20
.*
optimization finished, #iter = 312
nu = 0.000000
obj = -0.000024, rho = 2.173905
nSV = 23, nBSV = 0
Total nSV = 23
.*
optimization finished, #iter = 280
nu = 0.000000
obj = -0.000019, rho = 1.837889
nSV = 22, nBSV = 0
Total nSV = 22
.*
optimization finished, #iter = 354
nu = 0.000000
obj = -0.000023, rho = 1.673617
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 419
nu = 0.000000
obj = -0.000025, rho = 1.944385
nSV = 21, nBSV = 0
Total nSV = 21
*.*
optimization finished, #iter = 332
nu = 0.000000
obj = -0.000026, rho = 2.095987
nSV = 22, nBSV = 0
...*.*
optimization finished, #iter = 860
nu = 0.000000
obj = -0.000047, rho = 0.761

..*.*
optimization finished, #iter = 715
nu = 0.000000
obj = -0.000040, rho = 0.843918
nSV = 27, nBSV = 0
Total nSV = 27
..*..*
optimization finished, #iter = 836
nu = 0.000000
obj = -0.000038, rho = 0.714223
nSV = 25, nBSV = 0
Total nSV = 25
..*..*
optimization finished, #iter = 804
nu = 0.000000
obj = -0.000041, rho = 0.832895
nSV = 24, nBSV = 0
Total nSV = 24
..*..*
optimization finished, #iter = 1012
nu = 0.000000
obj = -0.000042, rho = 0.865224
nSV = 25, nBSV = 0
..*.*
optimization finished, #iter = 755
nu = 0.000001
obj = -0.000064, rho = -0.319862
nSV = 28, nBSV = 0
Total nSV = 28
..*.*
optimization finished, #iter = 715
nu = 0.000001
obj = -0.000061, rho = -0.186211
nSV = 27, nBSV = 0
Total nSV = 27
....*.....*
optimization finished, #iter = 1799
nu = 0.000001
obj = -0.000065, rho = 0.080657
nSV = 34, nBSV = 0
Total nSV = 34
..*.*
optimization finished, #iter = 661
nu = 0.000001
obj = -0.000060, rho = -0.376582
nSV = 26, nBSV = 0
Total nSV = 26
..*..*
optimization finished, #it

...

....*.*
optimization finished, #iter = 1401
nu = 0.000076
obj = -0.000091, rho = 0.959003
nSV = 34, nBSV = 0
.*.*
optimization finished, #iter = 510
nu = 0.000025
obj = -0.000023, rho = 2.158778
nSV = 22, nBSV = 0
Total nSV = 22
.*.*
optimization finished, #iter = 467
nu = 0.000022
obj = -0.000021, rho = 1.719926
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 423
nu = 0.000022
obj = -0.000021, rho = 1.878556
nSV = 23, nBSV = 0
Total nSV = 23
.*.*
optimization finished, #iter = 403
nu = 0.000024
obj = -0.000023, rho = 2.075472
nSV = 21, nBSV = 0
Total nSV = 21
.*.*
optimization finished, #iter = 470
nu = 0.000022
obj = -0.000021, rho = 1.756574
nSV = 24, nBSV = 0
Total nSV = 24
.*
optimization finished, #iter = 464
nu = 0.000020
obj = -0.000024, rho = 2.134785
nSV = 23, nBSV = 0
..*.*
optimization finished, #iter = 745
nu = 0.000066
obj = -0.000063, rho = 1.485128
nSV = 25, nBSV = 0
Total nSV = 25
....*.*
optimization finished, #iter = 969
nu = 0.000071
obj = -0.0

.*..*
optimization finished, #iter = 586
nu = 0.000023
obj = -0.000022, rho = 1.091252
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 391
nu = 0.000021
obj = -0.000020, rho = 1.103206
nSV = 19, nBSV = 0
Total nSV = 19
.*
optimization finished, #iter = 372
nu = 0.000022
obj = -0.000021, rho = 1.033407
nSV = 22, nBSV = 0
Total nSV = 22
.*.*
optimization finished, #iter = 574
nu = 0.000018
obj = -0.000022, rho = 1.075988
nSV = 22, nBSV = 0
..*.*
optimization finished, #iter = 756
nu = 0.000035
obj = -0.000034, rho = -0.083212
nSV = 27, nBSV = 0
Total nSV = 27
..*.*
optimization finished, #iter = 684
nu = 0.000037
obj = -0.000035, rho = -0.061750
nSV = 26, nBSV = 0
Total nSV = 26
.*.*
optimization finished, #iter = 400
nu = 0.000037
obj = -0.000035, rho = -0.086630
nSV = 26, nBSV = 0
Total nSV = 26
..*..*
optimization finished, #iter = 809
nu = 0.000035
obj = -0.000033, rho = -0.012043
nSV = 29, nBSV = 0
Total nSV = 29
...*..*
optimization finished, #iter = 1006
nu =

.*.*
optimization finished, #iter = 442
nu = 0.000028
obj = -0.000027, rho = -0.505944
nSV = 21, nBSV = 0
Total nSV = 21
*.*
optimization finished, #iter = 304
nu = 0.000027
obj = -0.000025, rho = -0.352301
nSV = 24, nBSV = 0
Total nSV = 24
*.*
optimization finished, #iter = 308
nu = 0.000027
obj = -0.000025, rho = -0.580675
nSV = 21, nBSV = 0
Total nSV = 21
.*
optimization finished, #iter = 358
nu = 0.000025
obj = -0.000029, rho = -0.534840
nSV = 22, nBSV = 0
.*
optimization finished, #iter = 327
nu = 0.000016
obj = -0.000015, rho = -1.194480
nSV = 18, nBSV = 0
Total nSV = 18
*.*
optimization finished, #iter = 204
nu = 0.000016
obj = -0.000015, rho = -1.054121
nSV = 16, nBSV = 0
Total nSV = 16
*.*
optimization finished, #iter = 245
nu = 0.000015
obj = -0.000014, rho = -1.085806
nSV = 16, nBSV = 0
Total nSV = 16
*.*
optimization finished, #iter = 311
nu = 0.000013
obj = -0.000013, rho = -1.391680
nSV = 18, nBSV = 0
Total nSV = 18
*
optimization finished, #iter = 167
nu = 0.000014
obj =

*.*
optimization finished, #iter = 248
nu = 0.000027
obj = -0.000026, rho = 1.604053
nSV = 18, nBSV = 0
Total nSV = 18
*.*
optimization finished, #iter = 262
nu = 0.000023
obj = -0.000027, rho = 1.480200
nSV = 18, nBSV = 0
*
optimization finished, #iter = 163
nu = 0.000013
obj = -0.000012, rho = 1.121980
nSV = 15, nBSV = 0
Total nSV = 15
*
optimization finished, #iter = 122
nu = 0.000011
obj = -0.000011, rho = 1.080239
nSV = 16, nBSV = 0
Total nSV = 16
*
optimization finished, #iter = 136
nu = 0.000013
obj = -0.000012, rho = 1.109913
nSV = 13, nBSV = 0
Total nSV = 13
*
optimization finished, #iter = 166
nu = 0.000011
obj = -0.000011, rho = 0.936631
nSV = 16, nBSV = 0
Total nSV = 16
*
optimization finished, #iter = 171
nu = 0.000013
obj = -0.000012, rho = 1.076759
nSV = 17, nBSV = 0
Total nSV = 17
*
optimization finished, #iter = 152
nu = 0.000010
obj = -0.000012, rho = 1.087867
nSV = 16, nBSV = 0
.*
optimization finished, #iter = 326
nu = 0.000021
obj = -0.000020, rho = 0.593476
nSV = 

*.*
optimization finished, #iter = 354
nu = 0.000027
obj = -0.000032, rho = -1.025208
nSV = 26, nBSV = 0
*
optimization finished, #iter = 43
nu = 0.000001
obj = -0.000000, rho = -2.146141
nSV = 8, nBSV = 0
Total nSV = 8
*
optimization finished, #iter = 43
nu = 0.000001
obj = -0.000000, rho = -2.146141
nSV = 8, nBSV = 0
Total nSV = 8
*
optimization finished, #iter = 31
nu = 0.000001
obj = -0.000000, rho = -2.022268
nSV = 6, nBSV = 0
Total nSV = 6
*
optimization finished, #iter = 25
nu = 0.000001
obj = -0.000000, rho = -2.134963
nSV = 8, nBSV = 0
Total nSV = 8
*
optimization finished, #iter = 39
nu = 0.000001
obj = -0.000000, rho = -2.113279
nSV = 5, nBSV = 0
Total nSV = 5
*
optimization finished, #iter = 45
nu = 0.000001
obj = -0.000000, rho = -2.147252
nSV = 8, nBSV = 0
.*
optimization finished, #iter = 341
nu = 0.000027
obj = -0.000026, rho = -1.199377
nSV = 20, nBSV = 0
Total nSV = 20
.*.*
optimization finished, #iter = 423
nu = 0.000028
obj = -0.000027, rho = -1.243739
nSV = 19, nBS

In [11]:
print("Classification Accuracy (80 dimensions C=1):",a)
print("Classification Accuracy (80 dimensions C=0.1):",b)
print("Classification Accuracy (80 dimensions C=0.01):",c)

Classification Accuracy (80 dimensions C=1): 98.51330203442879
Classification Accuracy (80 dimensions C=0.1): 98.51330203442879
Classification Accuracy (80 dimensions C=0.01): 98.43505477308294


In [12]:
d=SVM_calculation(train200,label,test200,test_label,1)
e=SVM_calculation(train200,label,test200,test_label,0.1)
f=SVM_calculation(train200,label,test200,test_label,0.01)

.*.*
optimization finished, #iter = 459
nu = 0.000000
obj = -0.000034, rho = -0.651078
nSV = 25, nBSV = 0
Total nSV = 25
..*.*
optimization finished, #iter = 610
nu = 0.000000
obj = -0.000034, rho = -0.516315
nSV = 28, nBSV = 0
Total nSV = 28
..*
optimization finished, #iter = 546
nu = 0.000000
obj = -0.000033, rho = -0.401867
nSV = 28, nBSV = 0
Total nSV = 28
.*.*
optimization finished, #iter = 412
nu = 0.000000
obj = -0.000032, rho = -0.443828
nSV = 28, nBSV = 0
Total nSV = 28
.*.*
optimization finished, #iter = 496
nu = 0.000000
obj = -0.000033, rho = -0.448201
nSV = 29, nBSV = 0
Total nSV = 29
.*.*
optimization finished, #iter = 500
nu = 0.000000
obj = -0.000036, rho = -0.504947
nSV = 31, nBSV = 0
.*.*
optimization finished, #iter = 420
nu = 0.000000
obj = -0.000036, rho = -0.053747
nSV = 26, nBSV = 0
Total nSV = 26
.*.*
optimization finished, #iter = 402
nu = 0.000000
obj = -0.000043, rho = 0.421518
nSV = 24, nBSV = 0
Total nSV = 24
.*.*
optimization finished, #iter = 522
nu = 0.0

*.*
optimization finished, #iter = 436
nu = 0.000000
obj = -0.000018, rho = 0.066383
nSV = 29, nBSV = 0
...*..*
optimization finished, #iter = 1100
nu = 0.000000
obj = -0.000034, rho = -0.795208
nSV = 35, nBSV = 0
Total nSV = 35
..*.*
optimization finished, #iter = 612
nu = 0.000000
obj = -0.000034, rho = -0.739330
nSV = 27, nBSV = 0
Total nSV = 27
..*.*
optimization finished, #iter = 713
nu = 0.000000
obj = -0.000032, rho = -0.778151
nSV = 30, nBSV = 0
Total nSV = 30
...*..*
optimization finished, #iter = 995
nu = 0.000000
obj = -0.000030, rho = -0.966246
nSV = 31, nBSV = 0
Total nSV = 31
..*.*
optimization finished, #iter = 621
nu = 0.000000
obj = -0.000034, rho = -0.863594
nSV = 30, nBSV = 0
Total nSV = 30
..*.*
optimization finished, #iter = 827
nu = 0.000000
obj = -0.000035, rho = -0.827139
nSV = 32, nBSV = 0
.*
optimization finished, #iter = 334
nu = 0.000000
obj = -0.000022, rho = -0.058116
nSV = 25, nBSV = 0
Total nSV = 25
.*
optimization finished, #iter = 369
nu = 0.000000
obj

.*.*
optimization finished, #iter = 421
nu = 0.000000
obj = -0.000041, rho = 0.819919
nSV = 29, nBSV = 0
Total nSV = 29
.*..*
optimization finished, #iter = 574
nu = 0.000000
obj = -0.000047, rho = 0.974518
nSV = 27, nBSV = 0
Total nSV = 27
.*.*
optimization finished, #iter = 594
nu = 0.000000
obj = -0.000048, rho = 0.993904
nSV = 30, nBSV = 0
.*.*
optimization finished, #iter = 412
nu = 0.000000
obj = -0.000039, rho = -0.263191
nSV = 27, nBSV = 0
Total nSV = 27
.*
optimization finished, #iter = 336
nu = 0.000000
obj = -0.000046, rho = 0.049556
nSV = 24, nBSV = 0
Total nSV = 24
.*
optimization finished, #iter = 349
nu = 0.000000
obj = -0.000047, rho = 0.129872
nSV = 24, nBSV = 0
Total nSV = 24
.*
optimization finished, #iter = 365
nu = 0.000000
obj = -0.000029, rho = 0.149585
nSV = 24, nBSV = 0
Total nSV = 24
.*.*
optimization finished, #iter = 412
nu = 0.000000
obj = -0.000045, rho = 0.407334
nSV = 26, nBSV = 0
Total nSV = 26
.*
optimization finished, #iter = 342
nu = 0.000000
obj = -

.....*.....*
optimization finished, #iter = 2018
nu = 0.000001
obj = -0.000080, rho = -1.059804
nSV = 44, nBSV = 0
Total nSV = 44
...*..*
optimization finished, #iter = 1058
nu = 0.000001
obj = -0.000080, rho = -1.011874
nSV = 40, nBSV = 0
Total nSV = 40
...*..*
optimization finished, #iter = 1025
nu = 0.000001
obj = -0.000086, rho = -1.087247
nSV = 38, nBSV = 0
Total nSV = 38
....*.*
optimization finished, #iter = 1135
nu = 0.000001
obj = -0.000088, rho = -1.024030
nSV = 41, nBSV = 0
Total nSV = 41
...*..*
optimization finished, #iter = 1212
nu = 0.000001
obj = -0.000090, rho = -1.074403
nSV = 44, nBSV = 0
..*
optimization finished, #iter = 556
nu = 0.000000
obj = -0.000040, rho = -0.236243
nSV = 35, nBSV = 0
Total nSV = 35
.*.*
optimization finished, #iter = 520
nu = 0.000000
obj = -0.000041, rho = -0.995567
nSV = 34, nBSV = 0
Total nSV = 34
.*.*
optimization finished, #iter = 464
nu = 0.000000
obj = -0.000040, rho = -0.658124
nSV = 33, nBSV = 0
Total nSV = 33
.*.*
optimization finis

..*
optimization finished, #iter = 555
nu = 0.000001
obj = -0.000052, rho = 1.211823
nSV = 32, nBSV = 0
Total nSV = 32
..*
optimization finished, #iter = 655
nu = 0.000000
obj = -0.000054, rho = 1.319370
nSV = 36, nBSV = 0
..*..*
optimization finished, #iter = 880
nu = 0.000000
obj = -0.000036, rho = -0.892919
nSV = 39, nBSV = 0
Total nSV = 39
..*..*
optimization finished, #iter = 921
nu = 0.000000
obj = -0.000041, rho = -0.995118
nSV = 36, nBSV = 0
Total nSV = 36
.*..*
optimization finished, #iter = 629
nu = 0.000000
obj = -0.000048, rho = -0.994123
nSV = 36, nBSV = 0
Total nSV = 36
..*.*
optimization finished, #iter = 680
nu = 0.000000
obj = -0.000044, rho = -0.933666
nSV = 35, nBSV = 0
Total nSV = 35
...*..*
optimization finished, #iter = 958
nu = 0.000000
obj = -0.000047, rho = -1.017165
nSV = 37, nBSV = 0
Total nSV = 37
..*.*
optimization finished, #iter = 934
nu = 0.000000
obj = -0.000048, rho = -1.011677
nSV = 41, nBSV = 0
.*
optimization finished, #iter = 300
nu = 0.000000
obj 

...

.*
optimization finished, #iter = 278
nu = 0.000023
obj = -0.000022, rho = -1.414506
nSV = 21, nBSV = 0
Total nSV = 21
*
optimization finished, #iter = 185
nu = 0.000023
obj = -0.000022, rho = -1.348169
nSV = 17, nBSV = 0
Total nSV = 17
*.*
optimization finished, #iter = 236
nu = 0.000025
obj = -0.000024, rho = -1.406226
nSV = 21, nBSV = 0
Total nSV = 21
*.*
optimization finished, #iter = 226
nu = 0.000022
obj = -0.000021, rho = -1.502011
nSV = 21, nBSV = 0
Total nSV = 21
*.*
optimization finished, #iter = 220
nu = 0.000022
obj = -0.000020, rho = -1.278291
nSV = 21, nBSV = 0
Total nSV = 21
*.*
optimization finished, #iter = 242
nu = 0.000020
obj = -0.000024, rho = -1.432087
nSV = 21, nBSV = 0
..*.*
optimization finished, #iter = 613
nu = 0.000068
obj = -0.000065, rho = -1.816329
nSV = 31, nBSV = 0
Total nSV = 31
.*..*
optimization finished, #iter = 626
nu = 0.000052
obj = -0.000049, rho = -1.722738
nSV = 30, nBSV = 0
Total nSV = 30
.*.*
optimization finished, #iter = 442
nu = 0.000067


.*
optimization finished, #iter = 352
nu = 0.000028
obj = -0.000026, rho = -0.250309
nSV = 26, nBSV = 0
Total nSV = 26
*.*
optimization finished, #iter = 242
nu = 0.000027
obj = -0.000026, rho = -0.134506
nSV = 25, nBSV = 0
Total nSV = 25
*.*
optimization finished, #iter = 293
nu = 0.000022
obj = -0.000027, rho = -0.209111
nSV = 28, nBSV = 0
.*
optimization finished, #iter = 264
nu = 0.000042
obj = -0.000040, rho = 0.293489
nSV = 27, nBSV = 0
Total nSV = 27
*.*
optimization finished, #iter = 242
nu = 0.000043
obj = -0.000041, rho = 0.275768
nSV = 25, nBSV = 0
Total nSV = 25
*.*
optimization finished, #iter = 277
nu = 0.000040
obj = -0.000038, rho = 0.292790
nSV = 24, nBSV = 0
Total nSV = 24
.*
optimization finished, #iter = 272
nu = 0.000040
obj = -0.000038, rho = 0.216960
nSV = 28, nBSV = 0
Total nSV = 28
.*.*
optimization finished, #iter = 544
nu = 0.000040
obj = -0.000038, rho = 0.378002
nSV = 29, nBSV = 0
Total nSV = 29
*.*
optimization finished, #iter = 322
nu = 0.000035
obj = -0.

*.*
optimization finished, #iter = 291
nu = 0.000029
obj = -0.000035, rho = 1.428320
nSV = 23, nBSV = 0
..*.*
optimization finished, #iter = 604
nu = 0.000050
obj = -0.000048, rho = -0.600149
nSV = 34, nBSV = 0
Total nSV = 34
..*..*
optimization finished, #iter = 778
nu = 0.000050
obj = -0.000048, rho = -0.713719
nSV = 29, nBSV = 0
Total nSV = 29
..*.*
optimization finished, #iter = 696
nu = 0.000046
obj = -0.000044, rho = -0.462738
nSV = 34, nBSV = 0
Total nSV = 34
..*.*
optimization finished, #iter = 683
nu = 0.000048
obj = -0.000045, rho = -0.743461
nSV = 29, nBSV = 0
Total nSV = 29
..*.*
optimization finished, #iter = 702
nu = 0.000050
obj = -0.000047, rho = -0.298796
nSV = 31, nBSV = 0
Total nSV = 31
..*.*
optimization finished, #iter = 756
nu = 0.000043
obj = -0.000052, rho = -0.642870
nSV = 33, nBSV = 0
.*.*
optimization finished, #iter = 551
nu = 0.000040
obj = -0.000038, rho = -0.597220
nSV = 31, nBSV = 0
Total nSV = 31
.*.*
optimization finished, #iter = 453
nu = 0.000040
obj

*
optimization finished, #iter = 151
nu = 0.000024
obj = -0.000023, rho = 0.412575
nSV = 17, nBSV = 0
Total nSV = 17
*.*
optimization finished, #iter = 241
nu = 0.000025
obj = -0.000024, rho = 0.484810
nSV = 17, nBSV = 0
Total nSV = 17
*
optimization finished, #iter = 148
nu = 0.000019
obj = -0.000018, rho = 0.436186
nSV = 17, nBSV = 0
Total nSV = 17
.*
optimization finished, #iter = 283
nu = 0.000025
obj = -0.000024, rho = 0.492209
nSV = 20, nBSV = 0
Total nSV = 20
*.*
optimization finished, #iter = 250
nu = 0.000025
obj = -0.000024, rho = 0.497432
nSV = 20, nBSV = 0
Total nSV = 20
*
optimization finished, #iter = 222
nu = 0.000020
obj = -0.000024, rho = 0.475707
nSV = 21, nBSV = 0
*.*
optimization finished, #iter = 224
nu = 0.000019
obj = -0.000018, rho = -0.825761
nSV = 17, nBSV = 0
Total nSV = 17
.*
optimization finished, #iter = 370
nu = 0.000019
obj = -0.000018, rho = -0.890706
nSV = 20, nBSV = 0
Total nSV = 20
.*
optimization finished, #iter = 368
nu = 0.000018
obj = -0.000017, 

Accuracy = 98.5915% (1260/1278) (classification)


In [13]:
print("Classification Accuracy (200 dimensions C=1):",d)
print("Classification Accuracy (200 dimensions C=0.1):",e)
print("Classification Accuracy (200 dimensions C=0.01):",f)

Classification Accuracy (200 dimensions C=1): 98.51330203442879
Classification Accuracy (200 dimensions C=0.1): 98.51330203442879
Classification Accuracy (200 dimensions C=0.01): 98.59154929577464


In [15]:
a=SVM_calculation(train40,label,test40,test_label,1)
b=SVM_calculation(train40,label,test40,test_label,0.1)
c=SVM_calculation(train40,label,test40,test_label,0.01)


..*.*
optimization finished, #iter = 629
nu = 0.000001
obj = -0.000052, rho = -0.230905
nSV = 19, nBSV = 0
Total nSV = 19
..*
optimization finished, #iter = 569
nu = 0.000001
obj = -0.000055, rho = -0.974605
nSV = 17, nBSV = 0
Total nSV = 17
..*..*
optimization finished, #iter = 915
nu = 0.000001
obj = -0.000068, rho = -0.482434
nSV = 20, nBSV = 0
Total nSV = 20
...*..*
optimization finished, #iter = 1127
nu = 0.000001
obj = -0.000063, rho = -0.799823
nSV = 21, nBSV = 0
Total nSV = 21
..*..*
optimization finished, #iter = 802
nu = 0.000001
obj = -0.000069, rho = -0.614737
nSV = 21, nBSV = 0
Total nSV = 21
..*..*
optimization finished, #iter = 1014
nu = 0.000001
obj = -0.000069, rho = -0.621305
nSV = 22, nBSV = 0
.*..*
optimization finished, #iter = 642
nu = 0.000001
obj = -0.000065, rho = -0.112009
nSV = 19, nBSV = 0
Total nSV = 19
..*..*
optimization finished, #iter = 784
nu = 0.000001
obj = -0.000064, rho = 0.120894
nSV = 20, nBSV = 0
Total nSV = 20
..*
optimization finished, #iter =

.*.*
optimization finished, #iter = 480
nu = 0.000001
obj = -0.000062, rho = 0.501956
nSV = 17, nBSV = 0
Total nSV = 17
.*.*
optimization finished, #iter = 549
nu = 0.000001
obj = -0.000063, rho = 0.477064
nSV = 18, nBSV = 0
..*...*
optimization finished, #iter = 1120
nu = 0.000001
obj = -0.000050, rho = 0.748720
nSV = 17, nBSV = 0
Total nSV = 17
...*.*
optimization finished, #iter = 883
nu = 0.000001
obj = -0.000058, rho = 1.103131
nSV = 21, nBSV = 0
Total nSV = 21
..*
optimization finished, #iter = 542
nu = 0.000000
obj = -0.000046, rho = 1.013263
nSV = 18, nBSV = 0
Total nSV = 18
...*.*
optimization finished, #iter = 833
nu = 0.000001
obj = -0.000058, rho = 1.071724
nSV = 21, nBSV = 0
Total nSV = 21
..*.*
optimization finished, #iter = 633
nu = 0.000001
obj = -0.000058, rho = 1.066424
nSV = 20, nBSV = 0
Total nSV = 20
...*.*
optimization finished, #iter = 1123
nu = 0.000000
obj = -0.000059, rho = 1.068819
nSV = 21, nBSV = 0
.*
optimization finished, #iter = 347
nu = 0.000000
obj = -

.......*.....*
optimization finished, #iter = 2999
nu = 0.000001
obj = -0.000085, rho = 1.175155
nSV = 25, nBSV = 0
...*...*
optimization finished, #iter = 1147
nu = 0.000000
obj = -0.000036, rho = 1.733644
nSV = 20, nBSV = 0
Total nSV = 20
..*..*
optimization finished, #iter = 801
nu = 0.000000
obj = -0.000027, rho = 1.407611
nSV = 23, nBSV = 0
Total nSV = 23
..*
optimization finished, #iter = 545
nu = 0.000000
obj = -0.000038, rho = 1.649317
nSV = 18, nBSV = 0
Total nSV = 18
.*.*
optimization finished, #iter = 546
nu = 0.000000
obj = -0.000027, rho = 1.302658
nSV = 20, nBSV = 0
Total nSV = 20
..**.*
optimization finished, #iter = 562
nu = 0.000000
obj = -0.000037, rho = 1.772785
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 548
nu = 0.000000
obj = -0.000038, rho = 1.584801
nSV = 19, nBSV = 0
.*.*
optimization finished, #iter = 462
nu = 0.000000
obj = -0.000026, rho = 1.504261
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 496
nu = 0.0000

......*
optimization finished, #iter = 1289
nu = 0.000001
obj = -0.000119, rho = -0.551970
nSV = 22, nBSV = 0
Total nSV = 22
....*.*
optimization finished, #iter = 1392
nu = 0.000001
obj = -0.000128, rho = -0.182164
nSV = 24, nBSV = 0
...*..*
optimization finished, #iter = 1083
nu = 0.000000
obj = -0.000042, rho = -0.460710
nSV = 19, nBSV = 0
Total nSV = 19
..*
optimization finished, #iter = 567
nu = 0.000000
obj = -0.000040, rho = -0.625850
nSV = 18, nBSV = 0
Total nSV = 18
..*...*
optimization finished, #iter = 1101
nu = 0.000000
obj = -0.000040, rho = -0.488362
nSV = 20, nBSV = 0
Total nSV = 20
.*
optimization finished, #iter = 334
nu = 0.000000
obj = -0.000032, rho = -0.335381
nSV = 17, nBSV = 0
Total nSV = 17
...*...*
optimization finished, #iter = 1215
nu = 0.000000
obj = -0.000042, rho = -0.428185
nSV = 18, nBSV = 0
Total nSV = 18
...*.*
optimization finished, #iter = 1058
nu = 0.000000
obj = -0.000042, rho = -0.461887
nSV = 19, nBSV = 0
*
optimization finished, #iter = 13
nu = 

.*
optimization finished, #iter = 244
nu = 0.000000
obj = -0.000035, rho = 1.294110
nSV = 13, nBSV = 0
Total nSV = 13
.*.*
optimization finished, #iter = 462
nu = 0.000000
obj = -0.000032, rho = 1.296562
nSV = 14, nBSV = 0
Total nSV = 14
.*
optimization finished, #iter = 313
nu = 0.000000
obj = -0.000032, rho = 1.336162
nSV = 16, nBSV = 0
Total nSV = 16
.*
optimization finished, #iter = 381
nu = 0.000000
obj = -0.000036, rho = 1.447770
nSV = 14, nBSV = 0
.*
optimization finished, #iter = 351
nu = 0.000000
obj = -0.000014, rho = 0.806295
nSV = 14, nBSV = 0
Total nSV = 14
*
optimization finished, #iter = 142
nu = 0.000000
obj = -0.000015, rho = 1.209716
nSV = 13, nBSV = 0
Total nSV = 13
*.*
optimization finished, #iter = 249
nu = 0.000000
obj = -0.000015, rho = 0.987655
nSV = 14, nBSV = 0
Total nSV = 14
*
optimization finished, #iter = 69
nu = 0.000000
obj = -0.000015, rho = 1.111649
nSV = 11, nBSV = 0
Total nSV = 11
*
optimization finished, #iter = 134
nu = 0.000000
obj = -0.000014, rho

...

..*.*
optimization finished, #iter = 632
nu = 0.000072
obj = -0.000069, rho = -0.531812
nSV = 19, nBSV = 0
Total nSV = 19
..*..*
optimization finished, #iter = 932
nu = 0.000062
obj = -0.000059, rho = -0.550491
nSV = 19, nBSV = 0
Total nSV = 19
...*.*
optimization finished, #iter = 902
nu = 0.000064
obj = -0.000061, rho = -0.619403
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 432
nu = 0.000069
obj = -0.000065, rho = -0.742363
nSV = 19, nBSV = 0
Total nSV = 19
..*.*
optimization finished, #iter = 678
nu = 0.000066
obj = -0.000062, rho = -0.468062
nSV = 19, nBSV = 0
Total nSV = 19
..*..*
optimization finished, #iter = 1014
nu = 0.000058
obj = -0.000069, rho = -0.621305
nSV = 22, nBSV = 0
..*...*
optimization finished, #iter = 1020
nu = 0.000067
obj = -0.000064, rho = -0.188498
nSV = 20, nBSV = 0
Total nSV = 20
.*.*
optimization finished, #iter = 466
nu = 0.000068
obj = -0.000065, rho = -0.003715
nSV = 17, nBSV = 0
Total nSV = 17
.*..*
optimization finished, #iter

...*.*
optimization finished, #iter = 803
nu = 0.000094
obj = -0.000089, rho = 0.790562
nSV = 20, nBSV = 0
Total nSV = 20
..*..*
optimization finished, #iter = 1001
nu = 0.000075
obj = -0.000090, rho = 0.859936
nSV = 24, nBSV = 0
.....*.*
optimization finished, #iter = 1156
nu = 0.000070
obj = -0.000067, rho = -1.182291
nSV = 19, nBSV = 0
Total nSV = 19
...*..*
optimization finished, #iter = 1059
nu = 0.000072
obj = -0.000069, rho = -1.164647
nSV = 20, nBSV = 0
Total nSV = 20
.....*...*
optimization finished, #iter = 1562
nu = 0.000065
obj = -0.000062, rho = -1.097649
nSV = 22, nBSV = 0
Total nSV = 22
......*.*
optimization finished, #iter = 1504
nu = 0.000077
obj = -0.000073, rho = -1.151037
nSV = 22, nBSV = 0
Total nSV = 22
.....*....*
optimization finished, #iter = 1863
nu = 0.000077
obj = -0.000073, rho = -1.165424
nSV = 23, nBSV = 0
Total nSV = 23
....*.*
optimization finished, #iter = 1249
nu = 0.000062
obj = -0.000074, rho = -1.099768
nSV = 20, nBSV = 0
.*.*
optimization finishe

...*.....*
optimization finished, #iter = 1557
nu = 0.000055
obj = -0.000052, rho = 1.170468
nSV = 21, nBSV = 0
Total nSV = 21
..*.*
optimization finished, #iter = 691
nu = 0.000062
obj = -0.000059, rho = 1.408514
nSV = 22, nBSV = 0
Total nSV = 22
..*.*
optimization finished, #iter = 929
nu = 0.000050
obj = -0.000059, rho = 1.409030
nSV = 22, nBSV = 0
.*
optimization finished, #iter = 299
nu = 0.000027
obj = -0.000026, rho = 1.646989
nSV = 17, nBSV = 0
Total nSV = 17
.*..*
optimization finished, #iter = 727
nu = 0.000024
obj = -0.000023, rho = 1.969035
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 459
nu = 0.000026
obj = -0.000025, rho = 1.730217
nSV = 18, nBSV = 0
Total nSV = 18
..*.*
optimization finished, #iter = 659
nu = 0.000026
obj = -0.000025, rho = 1.564212
nSV = 17, nBSV = 0
Total nSV = 17
.*.*
optimization finished, #iter = 432
nu = 0.000027
obj = -0.000026, rho = 1.835349
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 649
nu = 0

..*.*
optimization finished, #iter = 742
nu = 0.000037
obj = -0.000035, rho = 1.175871
nSV = 19, nBSV = 0
Total nSV = 19
.*.*
optimization finished, #iter = 421
nu = 0.000040
obj = -0.000038, rho = 1.653019
nSV = 19, nBSV = 0
Total nSV = 19
.*
optimization finished, #iter = 369
nu = 0.000040
obj = -0.000038, rho = 0.982912
nSV = 18, nBSV = 0
Total nSV = 18
.*.*
optimization finished, #iter = 519
nu = 0.000033
obj = -0.000040, rho = 1.209094
nSV = 19, nBSV = 0
...*.*
optimization finished, #iter = 837
nu = 0.000091
obj = -0.000087, rho = -0.438445
nSV = 19, nBSV = 0
Total nSV = 19
....*......*
optimization finished, #iter = 1924
nu = 0.000085
obj = -0.000081, rho = -0.553229
nSV = 26, nBSV = 0
Total nSV = 26
......*...*
optimization finished, #iter = 1779
nu = 0.000101
obj = -0.000096, rho = -0.217174
nSV = 20, nBSV = 0
Total nSV = 20
...*...*
optimization finished, #iter = 1319
nu = 0.000079
obj = -0.000075, rho = -0.366850
nSV = 22, nBSV = 0
Total nSV = 22
......*......*
optimization 

......*.*
optimization finished, #iter = 1419
nu = 0.000073
obj = -0.000070, rho = -0.720836
nSV = 20, nBSV = 0
Total nSV = 20
....*...*
optimization finished, #iter = 1518
nu = 0.000093
obj = -0.000088, rho = -0.557043
nSV = 22, nBSV = 0
Total nSV = 22
.......*....*
optimization finished, #iter = 2172
nu = 0.000080
obj = -0.000076, rho = -0.637286
nSV = 25, nBSV = 0
Total nSV = 25
....*..*
optimization finished, #iter = 1495
nu = 0.000075
obj = -0.000089, rho = -0.534017
nSV = 24, nBSV = 0
..*...*
optimization finished, #iter = 1055
nu = 0.000062
obj = -0.000060, rho = -1.014380
nSV = 22, nBSV = 0
Total nSV = 22
.*..*
optimization finished, #iter = 602
nu = 0.000061
obj = -0.000058, rho = -1.075973
nSV = 20, nBSV = 0
Total nSV = 20
..*.*
optimization finished, #iter = 665
nu = 0.000063
obj = -0.000060, rho = -1.031485
nSV = 22, nBSV = 0
Total nSV = 22
...*.*
optimization finished, #iter = 824
nu = 0.000053
obj = -0.000051, rho = -0.714057
nSV = 23, nBSV = 0
Total nSV = 23
...*...*
opt

In [16]:
print("Classification Accuracy (40 dimensions C=1):",d)
print("Classification Accuracy (40 dimensions C=0.1):",e)
print("Classification Accuracy (40 dimensions C=0.01):",f)

Classification Accuracy (40 dimensions C=1): 98.51330203442879
Classification Accuracy (40 dimensions C=0.1): 98.51330203442879
Classification Accuracy (40 dimensions C=0.01): 98.59154929577464
